In [1]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame

In [3]:
series_obj = Series(np.arrange(8), index=['row1', 'row2'. 'row3', 'row4', 'row5','row6', 'row7','row8'])

SyntaxError: invalid syntax (<ipython-input-3-038a4c7640d8>, line 1)

In [ ]:
np.random.seed(25)
DF_obj = DataFrame(np.randon.rand(36).reshape((6,6))

In [2]:
# MAKE SURE LATITUDE AND LONGITUDE ARE AT BEGINNING OF CSV
# import modules
import arcpy, os, csv
from datetime import datetime
# start timer
startTimer = datetime.now()
print("Imported arcpy, os, csv, and datatime modules.\n")
print(arcpy.ProductInfo())
## Set the Local Variables
#------------------------------------------------------------------------------------------------------#
# Change this to the path of your input feature class
inputfc = r"F:/Research and Analysis/LT Info Monitoring Data/live_data/live_data.gdb/Highway_Collisions_4"

# Change this to the path of your output FC
outfc = r"F:/GIS/GIS_DATA/Vector.sde/sde.SDE.LTINFO_Monitoring/sde.SDE.Highway_Collisions_1"

# feature dataset to unversion and register as version
fdata = r"F:/GIS/GIS_DATA/Vector.sde/sde.SDE.LTINFO_Monitoring"


#--------------------------------------------------------------------------------------------------------#

# unregister the sde feature class as versioned
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")

# set overwrite files envrionment setting to True
arcpy.env.overwriteOutput = True

# deletes all rows from the SDE feature class
arcpy.TruncateTable_management(outfc)
print ("\nDeleted all records in: {}\n".format(outfc))

# Get field objects from source FC
#
dsc = arcpy.Describe(inputfc)
fields = dsc.fields

# List all field names except the OID field
#
fieldnames = [field.name for field in fields if field.name != dsc.OIDFieldName]

# insert rows from Temporary feature class to SDE feature class
with arcpy.da.InsertCursor(outfc, fieldnames) as oCursor:
    count = 0
    with arcpy.da.SearchCursor(inputfc, fieldnames) as iCursor:
        for row in iCursor:
            oCursor.insertRow(row)
            count += 1
            if count % 1000 == 0:
                print("Inserting record {0} into SDE feature class".format(count))


# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")


# confirm feature class was created
print("\nUpdated " + outfc)

# report how long it took to run the script
endTimer = datetime.now() - startTimer
print ("\nTime it took to run this script: {}".format(endTimer))

Imported arcpy, os, csv, and datatime modules.

ArcInfo

Deleted all records in: F:/GIS/GIS_DATA/Vector.sde/sde.SDE.LTINFO_Monitoring/sde.SDE.Highway_Collisions_1

Inserting record 1000 into SDE feature class
Inserting record 2000 into SDE feature class
Inserting record 3000 into SDE feature class

Updated F:/GIS/GIS_DATA/Vector.sde/sde.SDE.LTINFO_Monitoring/sde.SDE.Highway_Collisions_1

Time it took to run this script: 0:00:40.839045


In [1]:
import pandas as pd
EcobjectCSV = 'C:/GIS\PROJECT/LakeTahoeWest/Data/Ecobject_LakeTahoeBasin.csv'

In [ ]:
def build_df_from_arcpy(table, index_col=None):
    """
    Build a pandas dataframe from an ArcGIS Feature Class.
    
    Uses the arcpy Search Cursor to loop over a feature class in pandas and
    generate a pandas dataframe. If the dataset is a feature class with a
    geometry it will calculate the length and the area before returning, and
    the geometry will be returned as well-known-text.
    
    :param table: The path to the feature class or table
    :type table: str
    :param index_col: A column to use as the dataframe index. If not supplied
                      for feature classes use the Object ID
    :type index_col: str
    :return: dataframe representation of the feature class. Note this is all in
             memory, so be careful with really big datasets!
    :rtype: pd.DataFrame
    """

    desc = arcpy.Describe(table)
    cursor = arcpy.SearchCursor(table)

    new_data = []

    for row in cursor:
        new_row = {}
        for field in desc.fields:
            new_row[field.aliasName or field.name] = row.getValue(field.name)
        new_data.append(new_row)

    try:
        if not index_col:
            index_col = desc.OIDFieldName

        df = pd.DataFrame(new_data).set_index(index_col)
        df["SHAPEArea"] = df[desc.shapeFieldName].apply(lambda g: g.area)
        df["SHAPELength"] = df[desc.shapeFieldName].apply(lambda g: g.length)
        df[desc.shapeFieldName] = df[desc.shapeFieldName].apply(lambda g: g.WKT)
    except AttributeError:
        # If this is a table in the datbase or on disk, in ArcGIS it won't have
        # either an OID field, nor a geometry
        pass

    return df

In [2]:
df = pd.read_csv(EcobjectCSV, low_memory=False)

In [3]:
df.dtypes

OBJECTID                       int64
ECOREGION_DOMAIN             float64
ECOREGION_DIVISION           float64
ECOREGION_PROVINCE            object
ECOREGION_SECTION             object
ECOREGION_SUBSECTION          object
CALVEGZONE                   float64
TILE                          object
COVERTYPE                     object
REGIONAL_DOMINANCE_TYPE_1     object
OS_TREE_DIAMETER_CLASS_1     float64
REGIONAL_DOMINANCE_TYPE_2     object
OS_TREE_DIAMETER_CLASS_2     float64
REGIONAL_DOMINANCE_TYPE_3    float64
CON_CFA                      float64
HDW_CFA                      float64
SHB_CFA                      float64
HEB_CFA                      float64
DATA_SOURCE                    int64
R05_DATA_SOURCE               object
SOURCE_DATE                  float64
MAP_UPDATE_CAUSE              object
CAUSE_DATE                   float64
REV_DATE                     float64
TOTAL_TREE_CFA               float64
TREE_CFA_CLASS_1             float64
PROD                          object
C

In [4]:
df.head()

,OBJECTID,ECOREGION_DOMAIN,ECOREGION_DIVISION,ECOREGION_PROVINCE,ECOREGION_SECTION,ECOREGION_SUBSECTION,CALVEGZONE,TILE,COVERTYPE,REGIONAL_DOMINANCE_TYPE_1,...,MinTemp,MaxTemp,MeanTemp,CWHR,C_Area,C_Post_Acq,C_Date,Precip_Yr,Shape_Length,Shape_Area
0,1,200.0,260.0,M261,M261E,M261El,3.0,17B,CON,JP,...,-5,25,8,JPN2,25,30,2011,NaN,204.946839,1316.040262
1,2,200.0,260.0,M261,M261E,M261El,3.0,17B,CON,JP,...,-6,25,8,JPN4P,31,30,2016,NaN,126.696626,847.193514
2,3,200.0,260.0,M261,M261E,M261El,3.0,17B,CON,JP,...,-6,25,8,JPN4D,61,37,2016,NaN,137.972550,909.337417
3,4,200.0,260.0,M261,M261E,M261El,3.0,17B,CON,MF,...,-6,24,8,SMC3P,0,0,NaN,NaN,28.748860,54.180288
4,5,200.0,260.0,M261,M261E,M261El,3.0,17B,CON,JP,...,-5,25,9,JPN3S,0,0,NaN,NaN,25.564590,48.619957


In [5]:
df.dropna()

,OBJECTID,ECOREGION_DOMAIN,ECOREGION_DIVISION,ECOREGION_PROVINCE,ECOREGION_SECTION,ECOREGION_SUBSECTION,CALVEGZONE,TILE,COVERTYPE,REGIONAL_DOMINANCE_TYPE_1,...,MinTemp,MaxTemp,MeanTemp,CWHR,C_Area,C_Post_Acq,C_Date,Precip_Yr,Shape_Length,Shape_Area


In [ ]:
whr = df.plot.bar(x='WHRTYPE',y='Acres')
whr

In [6]:
df.index = df.WHRTYPE

In [ ]:
whr = df.Acres.plot.barh()
whr

In [ ]:
dtype=  {'OBJECTID':'float64', 
                                       'COVERTYPE':'object',
                                       'REGIONAL_DOMINANCE_TYPE_1':'object',
                                       'OS_TREE_DIAMETER_CLASS_1':'object',
                                       'CON_CFA':'object',
                                       'HDW_CFA':'object',
                                       'SHB_CFA':'object',
                                       'HEB_CFA':'object', 
                                       'TOTAL_TREE_CFA':'object',
                                       'TREE_CFA_CLASS_1':'object',
                                       'PROD':'object',
                                       'CANOPYSTRUCTURE':'object',
                                       'REFORESTATION_STATUS':'object',
                                       'ORIGIN_YEAR':'float64',
                                       'WHRLIFEFORM':'object',
                                       'WHRTYPE':'object',
                                       'WHRSIZE':'object',
                                       'WHRDENSITY':'object',
                                       'UniqueID':'float64',
                                       'CH_95_M':'float64',
                                       'CH_95_FT':'float64',
                                       'CH_Mean_FT':'float64',
                                       'Can_Cov':'float64',
                                       'CC2_8':'float64',
                                       'CC8_16':'float64',
                                       'CC16_32':'float64',
                                       'CC32None':'float64',
                                       'Avg_Slope':'float64',
                                       'LMU':'object',
                                       'pTree_Count':'float64',
                                       'Acres':'float64',
                                       'pTree_Count_acre':'float64',
                                       'SpatialVar':'object',
                                       'Top_Succes':'object',
                                       'Strata':'object',
                                       'Strata_Dis':'object',
                                       'Ownership':'object',
                                       'Development':'object',
                                       'pBA_acre':'float64',
                                       'QMD':'float64',
                                       'CWHR_CC':'object',
                                       'CWHR_Size':'object',
                                       'CCFireSum':'float64',
                                       'CC_Mult_FRI':'float64',
                                       'LAFRI':'object',
                                       'DBH_10_20_ct':'float64',
                                       'DBH_20_30_ct':'float64',
                                       'DBH_30p_ct':'float64',
                                       'DBH_10_20_TPA':'float64',
                                       'DBH_20_30_TPA':'float64',
                                       'DBH_30p_TPA':'float64',
                                       'Water_Bal':'float64',
                                       'Elev_Ft':'int64',
                                       'pTotalCubicVol_CF':'float64',
                                       'pTotalCubicVol_CF_acre':'float64',
                                       'pGrossScribnerVol_BF':'float64',
                                       'pGrossScribnerVol_BF_acre':'float64',
                                       'pGrossMerchVol_CF':'float64',
                                       'pGrossMerchVol_CF_acre':'float64',
                                       'pGrossSecondVol_CF':'float64',
                                       'pGrossSecondVol_CF_acre':'float64',
                                       'Live_AGB':'float64',
                                       'Live_AGC':'float64',
                                       'Live_AGC_hectare':'float64',
                                       'AnnualCWD':'float64',
                                       'AnnualAET':'float64',
                                       'AnnualPET':'float64',
                                       'AnnualPrecip':'float64',
                                       'MaxRadiation':'float64',
                                       'AnnualRain':'float64',
                                       'AnnualSnow':'float64',
                                       'MeanSoilWater':'float64',
                                       'AnnualRunoff':'float64',
                                       'MinTemp':'float64',
                                       'MaxTemp':'float64',
                                       'MeanTemp':'float64',
                                       'CWHR':'object',
                                       'C_Area':'float64',
                                       'C_Post_Acq':'float64',
                                       'C_Date':'object',
                                       'Precip_Yr':'float64',
                                       'Shape_Length':'float64',
                                       'Shape_Area':'float64'},